Shopping Log - Activity Report
https://plum.greenehillfood.coop/civicrm/report/instance/103?reset=1&output=criteria

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import datetime
import itertools
from datetimerange import DateTimeRange
import pickle
import sqlalchemy

In [2]:
#os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI')
#os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI/member_shopping/civi_member_shopping')
#/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports
os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports')
print(os.getcwd())

/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports


In [3]:
os.listdir()

['Report_20230925.csv',
 'Report_20230730-1232.csv',
 'Report_20230709-1156.csv',
 'Report_20230910-1405.csv',
 'memberShopping_20240101.csv',
 'memberShopping_0827.csv',
 'memberShopping_20231203.csv',
 'memberShopping_20230925.csv']

## Consider doing QA between several files

In [4]:
shop = pd.read_csv('./memberShopping_20240101.csv')

In [5]:
#rename fields
shop.columns = [i.replace(' ','_') for i in shop.columns]

In [6]:
shop['Activity_Date'] = pd.to_datetime(shop['Activity_Date'], format = "%Y-%m-%d %H:%M")

In [7]:
shop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3466 entries, 0 to 3465
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Assignee_Name    0 non-null      float64       
 1   Target_Name      3466 non-null   object        
 2   Source_Email     0 non-null      float64       
 3   Assignee_Email   0 non-null      float64       
 4   Target_Email     3466 non-null   object        
 5   Activity_Type    3466 non-null   object        
 6   Subject          3466 non-null   object        
 7   Activity_Date    3466 non-null   datetime64[ns]
 8   Activity_Status  3466 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 243.8+ KB


Review some characteristics of the dataset

In [8]:
print(shop['Activity_Type'].value_counts())
print(shop['Activity_Status'].value_counts())
print(shop['Subject'].value_counts())
#shop['Activity Type'].value_counts()

Shop    13138
Name: Activity_Type, dtype: int64
Completed    13138
Name: Activity_Status, dtype: int64
Checked in to shop via email      9924
Checked in to shop via barcode    3214
Name: Subject, dtype: int64


In [8]:
#date range
shop['Activity_Date'].max()

Timestamp('2023-12-31 15:50:00')

Fields of interest: Target_Name, Target_Email, Activity_Type, Subject, Activity_Date

In [9]:
shop['ingest_date'] = datetime.datetime.today()

## Import Current Member roster from CIVI

In [3]:
active_mem = pd.read_csv('./activeDetailedReportCIVI_11012023.csv')

In [4]:
active_mem.columns = [i.replace(' ','_') for i in active_mem.columns]

In [5]:
active_mem['Start_Date'] = pd.to_datetime(active_mem['Start_Date'], format = "%Y-%m-%d")

In [6]:
#establish connection to db and import the table
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'

url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database)

engine = sqlalchemy.create_engine(url)
#, connect_args={'options': '-csearch_path=schema_name'}, isolation_level="AUTOCOMMIT"

#load the new table
with engine.connect() as conn:
    active_mem.to_sql('active_roster', con=engine, if_exists='replace', index=False)

In [10]:
#establish connection to db and import the table
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'

url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database)

engine = sqlalchemy.create_engine(url)
#, connect_args={'options': '-csearch_path=schema_name'}, isolation_level="AUTOCOMMIT"

#load the new table
with engine.connect() as conn:
    shop.loc[:,['Target_Name', 'Target_Email', 'Activity_Type', 'Subject', 'Activity_Date','ingest_date']].to_sql('shop_log', con=engine, if_exists='replace', index=False)